# Twitter Spam Detection

## Feature Extraction

We will begin our ML pipeline with feature extraction in this homework, as the dataset is good enough and does not need any special pre-processing. As usual, we will load the dataset into memory, extract samples features into vectors and split the dataset into training and test sets.

In [22]:
import pandas as pd

## [ TODO ]
# 1) Load all data from `twitter-mini.csv` into variable `data`
# 2) Preview first few samples
data = pd.read_csv("twitter-mini.csv")
data.head()

,text,label
0,@baydiangirl it definitely moved some church f...,0
1,@suebob Looks like #TXlege made up new rules f...,0
2,RT @saleemchat: #WJChat #A1B 5 yrs: People ski...,0
3,@SajidaBalouch Huhhhhh,0
4,@Dukester_94 im the lad of all bibles,0


In lab 2 we have learned how to use the `CountVectorizer` feature extractor, which counts the occurance of each token (letter here) in a sample (Twitter message text here). This time, we will also try `TfidfVectorizer` and see which feature extractor performs better. Here we will introduce the algorithm it implemented, **TF-IDF**:

> **TF-IDF** is the abbreviation for **term frequency–inverse document frequency**, and is defined as the product of the two frequency. The most classical version of TF-IDF is:
>
> $$
tf-idf(t) = tf(t, d) \cdot idf(t, D) = \frac{f_{t,d}}{|d|} \log \frac{|D|}{|D_t|}
$$
>
> Where $t$ is the term, $D$ is the collection of all documents, $d$ is one of the document that contains $t$ and $D_t$ is the subset of documents that contains term $t$. The advantage of TF-IDF is that frequent and useless words, such as "the", "a" and "an" will receive lower weight, since they appear in almost all documents. On the other hand, unique yet frequent (in any $d \in D_t$) words will get higher weight. Research has shown that TF-IDF is easy to implement and performs relatively well.

Before we run these feature extractors on our dataset, we need to convert each message into a sequence of terms. Since Twitter messages often contain emojis, abbreviations, hashtags and other composition of characters, we can't use the word-level analyzer as we have done in lab 2. Instead, we will generate **character N-grams** from these messages.

> A **character N-gram** consists of N consecutive characters. Here we show all character 1, 2, 3 and 4-grams for word "cold":
> <img src="attachment:char-ngram.jpg" width="50%" />

Both `CountVectorizer` and `TfidfVectorizer` provide support for character N-grams. Refer to `sklearn`'s documents to build a character N-gram analyzer.

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
texts = data["text"].values
# 1) Extract features of all messages into `feat1` with `CountVectorizer` using CHARACTER trigrams
# 2) Repeat with `TfidfVectorizer` and save generated features as `feat2`
print("Extracting features with `CountVectorizer`...")
cv = CountVectorizer(analyzer="char", ngram_range=(3, 3))
feat1 = cv.fit_transform(texts)

print("Extracting features with `TfidfVectorizer`...")
tfidf = TfidfVectorizer(analyzer="char", ngram_range=(3, 3))
feat2 = tfidf.fit_transform(texts)

print("Completed.")

Extracting features with `CountVectorizer`...
Extracting features with `TfidfVectorizer`...
Completed.


We will also try combining two kinds of feature generated above into a single feature. We will use `scipy.sparse.hstack` to concatenate corresonding features for each sample. Remember, concatenated features matrix should have same amount of samples as `feat1` and `feat2`.

In [24]:
# Concatenate token count features and TF-IDF features on the feature dimension,
# name the combined feature matrix `feat_comb`.
from scipy.sparse import hstack

feat_comb = hstack([feat1, feat2])

Finally, let's take a look at the labels. In our dataset, 0 represents messages from real Twitter users and 1 represents spam messages.

In [25]:
y = data["label"].values

## Training

As usual, we will start by doing a 80% / 20% training-test split. Like lab 2, we suggest you to **use the same random seed for all splits** for comparable and reproducible results.

In [26]:
from sklearn.model_selection import train_test_split
# 1) Split token count features into 80/20 training and test sets
# 2) Split TF-IDF features into 80/20 training and test sets
# 3) Split combined features into 80/20 training and test sets

# Token count features
print("Spliting token count features...")
X1_train, X1_test, y_train, y_test = train_test_split(
    feat1, y, test_size=0.2, random_state=42)

# TF-IDF features
print("Spliting TF-IDF features...")
X2_train, X2_test, _, _ = train_test_split(
    feat2, y, test_size=0.2, random_state=42)

# Combined features
print("Spliting combined features...")
Xc_train, Xc_test, _, _ = train_test_split(
    feat_comb, y, test_size=0.2, random_state=42)

print("Completed.")

Spliting token count features...
Spliting TF-IDF features...
Spliting combined features...
Completed.


We are now ready to train **binary classifiers** for spam message detection. Apart from the `LogisticRegression` classifier you have already seen and used during Lab 1, we will try a few more classifiers:

* `MultinomialNB` (Naive Bayes):  
  This classifier is based on the **Naive Bayes** assumption that, *given the class label*, individual features are conditionally independent.  
  For text classification, `MultinomialNB` is used with **token count** or **TF-IDF-like** features.

  It models the probability of a message belonging to a class $ y \in \{\text{spam}, \text{ham}\} $ as:

  $$
  P(y \mid \mathbf{x}) \propto P(y)\prod_{j=1}^{d} P(x_j \mid y)
   $$

  where $\mathbf{x} = (x_1, x_2, \dots, x_d)$ is the feature vector (e.g., word counts), and $P(x_j \mid y)$ is learned from training data.  
  The predicted label is chosen using:

   $$
  y_{pred} = \arg\max_y \; P(y \mid \mathbf{x})
   $$

  In practice, the computation is done in the **log domain** to avoid numerical underflow:

  $$
  y_{pred} = \arg\max_y \left( \log P(y) + \sum_{j=1}^{d} x_j \log P(x_j \mid y) \right)
  $$

* `LinearSVC`:  
  This classifier is based on **linear SVM**, a popular algorithm for binary classification. The linear SVM algorithm treats the feature vector of each sample as a point in a linear space, and tries to find a hyperplane that separates points of the two classes as well as possible.  

  In practice, we optimize a **squared hinge loss with regularization**, since the features may not be perfectly linearly separable:

   $$
  l_{hinge} = \max(0, y(1 - \mathbf{x}^T \mathbf{w}))^2 + \beta ||\mathbf{w}||_2^2
   $$

  Similarly, the classification result is determined by:

  $$
  y_{pred} = sign(1 - \mathbf{x}^T \mathbf{w})
 $$

We have three kinds of features (**token count**, **TF-IDF**, and **combined**) and three kinds of classifiers. Here we will only try three combinations and compare their performance:

* `MultinomialNB` on token count features  
* `LogisticRegression` on TF-IDF features  
  - Try increasing the number of iterations if a warning of non-convergence pops up  
* `LinearSVC` on combined features  
  - For linear SVM, you can ignore the non-convergence warning as it does not affect performance  


In [27]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
# 1) Train a `MultinomialNB` classifier called `model1` on token count training set
# 2) Train a `LogisticRegression` classifier called `model2` on TF-IDF training set
# 3) Train a `LinearSVC` classifier called `model_comb` on combined features training set

print("Training Naive Bayes classifier on token count training set...")
model1 = MultinomialNB()
model1.fit(X1_train, y_train)

print("Training logistic regression classifier on TF-IDF training set...")
model2 = LogisticRegression(max_iter=1000)
model2.fit(X2_train, y_train)

print("Training SVM classifier on combined features training set...")
model_comb = LinearSVC()
model_comb.fit(Xc_train, y_train)

print("Completed.")

Training Naive Bayes classifier on token count training set...
Training logistic regression classifier on TF-IDF training set...
Training SVM classifier on combined features training set...
Completed.


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Evaluation

Finally, it's time to evaluate our models. You probably noticed that we have repeated similar code for different feature extraction and classification models. To avoid that, we will use `for` loops to make evaluation simpler:

In [28]:
features = [("Token count", feat1), ("TF-IDF", feat2), ("Combined", feat_comb)]
models = [("Naive Bayes", model1), ("Logistic", model2), ("SVM", model_comb)]

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# Complete missing code in the loop body
for i, ((feat_name, feat), (model_name, model)) in enumerate(zip(features, models)):
    # Begin of metrics for combination
    print("====================")
    print(f"{i+1}. {model_name} on {feat_name}:\n")

    # 1) Split features into training and test set
    # (Note: you should use the same random number seed as the one in the training-test split above)
    X_train, X_test, y_train, y_test = train_test_split(
        feat, y, test_size=0.2, random_state=42
    )

    # 2) Predict labels for training data
    pred_train = model.predict(X_train)

    # 3) Print metrics for training set (precision, recall and F1 metrics)
    print("Training metrics:", classification_report(y_train, pred_train))
    # Confusion matrix
    print("Training confusion matrix:\n", confusion_matrix(y_train, pred_train), "\n")
    # Accuracy
    print("Train accuracy:", accuracy_score(y_train, pred_train), "\n")

    # 4) Predict labels for test data
    pred_test = model.predict(X_test)

    # 5) Print metrics for test set (precision, recall and F1 metrics)
    print("Test metrics:", classification_report(y_test, pred_test))
    # Confusion matrix
    print("Test Confusion Matrix:\n", confusion_matrix(y_test, pred_test), "\n")
    # Accuracy
    print("Test Accuracy:", accuracy_score(y_test, pred_test))

    # End of metrics for combination
    print("====================\n")

1. Naive Bayes on Token count:

Training metrics:               precision    recall  f1-score   support

           0       0.92      0.99      0.95     19978
           1       0.98      0.91      0.95     20022

    accuracy                           0.95     40000
   macro avg       0.95      0.95      0.95     40000
weighted avg       0.95      0.95      0.95     40000

Training confusion matrix:
 [[19694   284]
 [ 1748 18274]] 

Train accuracy: 0.9492 

Test metrics:               precision    recall  f1-score   support

           0       0.92      0.98      0.95      5022
           1       0.98      0.91      0.94      4978

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000

Test Confusion Matrix:
 [[4939   83]
 [ 456 4522]] 

Test Accuracy: 0.9461

2. Logistic on TF-IDF:

Training metrics:               precision    recall  f1-score   support

           0       0.94

# Questions
Please answer the following questions. You can just write down the answer below each question in this cell.


1. Please compute `tf-idf` of word-level **bigrams** for the following three messages **by hand**. You should use the TF-IDF definition introduced in class.

  * Message 1: "vegetables are good for health"
  * Message 2: "fruits are good source of vitamins"
  * Message 3: "proteins are good for health"

  Compute and sort intermediate and final results for **all occurring bigrams in the following order**:
  
  ```
  ["vegetables are", "are good", "good for", "for health", "fruits are", "good source", "source of", "of vitamins", "proteins are"]
  ```
  
  Then complete the following questions:

  1. Term Frequency (`tf`) of **bigrams for message 2**.
  
  *[0, 1, 0, 0, 1, 1, 1, 1, 0]*

  2. Inverse Document Frequency (`idf`) of **bigrams for message 2**.
    
  *[log3, 0, log(3/2), log(3/2), log3, log3, log3, log3, log3]*

  3. `tf-idf` of **bigrams for message 2**.
    
  *[0,0,0,0,log3,log3,log3,log3,0]*

  4. Is the `tf-idf` value for the term "are good" equal to 0? If it's 0, can you explain why it makes sense to have 0 for that term?
    
  *Yes it is zero. This make sense because it appears in all the documents, making it useless in contributing to the distinguishing power.*

  5. If we consider unigram (1-gram) based bag of words, how many words in these three messages would be assigned a 0 value tf-idf? And please explain why.
    
  *"Are" and "Good" are going to be assigned a 0 value, because they appeared in every single sentence, making them useless in distinguishing the sentences apart from a statistical standpoint.*



2. What does `support` mean in the `classification_report`?

*Support is the number of true samples for each class in the dataset being evaluated.*

3. If the training data is highly unbalanced, is `accuracy` a good metric for the classifiers' performance? Does a high accuracy necessarily means a high f1-score? Compare `accuracy` with `f1-score`. Based on your criterion, which type of classifier is better? Please give some justifications to support your answer.

*No, when data is imbalanced, accuracy is not a good metric for the classifiers' performance. A high accuracy score does not translate to a high F1 score. Say for example with a dataset with 95 correct and 5 wrong, and the model predicted 100 to be correct. This scenario makes accuracy 95%, which seems pretty good, but the recall for the wrong is 0, and so the F1 score would be 0 too, which means this model is not predicting well.*

4. For most cases, the amount of bad data is much less than good data. How can you mitigate the data imbalance issue?

*Two ways to do this: reducing the amount of good data (undersampling), or sythesize the bad data (oversampling).*

## References

1. CRESCI-2017 Dataset: https://botometer.osome.iu.edu/bot-repository/datasets.html
2. TF-IDF: https://en.wikipedia.org/wiki/Tf%E2%80%93idf
3. N-grams: https://en.wikipedia.org/wiki/N-gram
4. `scikit-learn` documentation: https://scikit-learn.org/stable/modules/classes.html
5. Ridge regression: https://en.wikipedia.org/wiki/Tikhonov_regularization
6. SVM: https://en.wikipedia.org/wiki/Support-vector_machine
7. Lasso: https://en.wikipedia.org/wiki/Lasso_(statistics)
8. ElasticNet: https://en.wikipedia.org/wiki/Elastic_net_regularization
9. Kernel method: https://en.wikipedia.org/wiki/Kernel_method